In [ ]:
# TU Q098_A · Toy Anthropocene trajectories
# Single-cell Colab script (offline, no API key required)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# -----------------------------
# 1. High-level description
# -----------------------------

print("TU Q098_A · Toy Anthropocene trajectories")
print("--------------------------------------------------")
print("This notebook simulates three tiny Anthropocene scenarios")
print("with three state variables:")
print("  X_t: economic output / energy use")
print("  E_t: environmental load / cumulative impact")
print("  C_t: adaptive capacity / governance strength")
print()
print("We define a simple safe operating region in the (E, C) plane and")
print("compute a scalar tension observable T_anthro for each scenario.")
print("No API key is needed. Everything is fully offline.\n")


# -----------------------------
# 2. Safe operating region
# -----------------------------

# Safe region is a simple rectangle in the (E, C) plane.
safe_region = {
    "E_min": 0.2,
    "E_max": 1.0,
    "C_min": 0.4,
    "C_max": 1.0,
}

print("Safe operating region in (E, C):")
print(f"  E between {safe_region['E_min']} and {safe_region['E_max']}")
print(f"  C between {safe_region['C_min']} and {safe_region['C_max']}\n")


# -----------------------------
# 3. Scenario configuration
# -----------------------------

# Each scenario has:
# - id: short code
# - name: human-readable label
# - description: one-line explanation
# - params: update rule parameters
#
# The update rules will use these parameters:
#   growth_rate          : baseline growth of X
#   env_sensitivity      : how strongly X increases E
#   env_decay            : natural decay rate of E
#   adapt_gain           : how much C grows under moderate stress
#   stress_sensitivity   : how much C is eroded under high stress
#   fatigue              : slow decay of C over time
#   policy_shift_step    : time step where parameters change (optional)
#   *_post               : parameter values after the policy shift (optional)

T_STEPS = 60  # number of time steps for all scenarios

scenarios = [
    {
        "id": "Q098A_S1",
        "short_name": "runaway",
        "name": "Runaway exploitation",
        "description": "High growth, weak regulation; environmental load runs away.",
        "params": {
            "X0": 1.0,
            "E0": 0.25,
            "C0": 0.55,
            "growth_rate": 0.08,
            "env_sensitivity": 0.06,
            "env_decay": 0.01,
            "adapt_gain": 0.015,
            "stress_sensitivity": 0.09,
            "fatigue": 0.010,
            "policy_shift_step": None,  # no policy shift
        },
    },
    {
        "id": "Q098A_S2",
        "short_name": "balanced",
        "name": "Balanced policy",
        "description": "Moderate growth, active adaptation; stays closer to the safe region.",
        "params": {
            "X0": 1.0,
            "E0": 0.3,
            "C0": 0.6,
            "growth_rate": 0.05,
            "env_sensitivity": 0.045,
            "env_decay": 0.015,
            "adapt_gain": 0.020,
            "stress_sensitivity": 0.050,
            "fatigue": 0.010,
            "policy_shift_step": None,
        },
    },
    {
        "id": "Q098A_S3",
        "short_name": "overshoot",
        "name": "Overshoot then correction",
        "description": "Starts like runaway, then policy correction reduces pressure.",
        "params": {
            "X0": 1.0,
            "E0": 0.28,
            "C0": 0.55,
            # pre-shift parameters (similar to runaway)
            "growth_rate": 0.075,
            "env_sensitivity": 0.058,
            "env_decay": 0.010,
            "adapt_gain": 0.016,
            "stress_sensitivity": 0.085,
            "fatigue": 0.010,
            # policy shift configuration
            "policy_shift_step": 25,  # at t >= 25, we apply "policy corrections"
            "growth_rate_post": 0.040,
            "env_sensitivity_post": 0.040,
            "env_decay_post": 0.018,
            "adapt_gain_post": 0.024,
            "stress_sensitivity_post": 0.055,
            "fatigue_post": 0.010,
        },
    },
]

print("Configured scenarios:")
for sc in scenarios:
    print(f"  - {sc['short_name']}: {sc['name']}")
print("")


# -----------------------------
# 4. Update rules and helpers
# -----------------------------

def simulate_scenario(params, safe, n_steps=60):
    """
    Simulate one Anthropocene toy scenario.

    Args:
        params: dict with initial conditions and update parameters.
        safe: dict describing the safe operating region.
        n_steps: number of time steps.

    Returns:
        dict with arrays X, E, C of length n_steps.
    """
    X = np.zeros(n_steps)
    E = np.zeros(n_steps)
    C = np.zeros(n_steps)

    X[0] = params["X0"]
    E[0] = params["E0"]
    C[0] = params["C0"]

    for t in range(n_steps - 1):
        # Select parameter set (pre- or post-policy shift)
        if params.get("policy_shift_step") is not None and t >= params["policy_shift_step"]:
            growth_rate = params.get("growth_rate_post", params["growth_rate"])
            env_sensitivity = params.get("env_sensitivity_post", params["env_sensitivity"])
            env_decay = params.get("env_decay_post", params["env_decay"])
            adapt_gain = params.get("adapt_gain_post", params["adapt_gain"])
            stress_sensitivity = params.get("stress_sensitivity_post", params["stress_sensitivity"])
            fatigue = params.get("fatigue_post", params["fatigue"])
        else:
            growth_rate = params["growth_rate"]
            env_sensitivity = params["env_sensitivity"]
            env_decay = params["env_decay"]
            adapt_gain = params["adapt_gain"]
            stress_sensitivity = params["stress_sensitivity"]
            fatigue = params["fatigue"]

        # Check comfort / stress conditions based on current E and C
        in_safe = (
            (E[t] >= safe["E_min"]) and (E[t] <= safe["E_max"]) and
            (C[t] >= safe["C_min"]) and (C[t] <= safe["C_max"])
        )
        high_stress = E[t] > safe["E_max"]
        moderate_band = (E[t] >= safe["E_min"]) and (E[t] <= 0.9 * safe["E_max"])

        # 4.1 Economic / energy variable X
        # Growth is stronger when conditions are comfortable,
        # weaker when the system is outside the safe region.
        if in_safe:
            growth_factor = 1.0
        elif high_stress:
            growth_factor = 0.4
        else:
            growth_factor = 0.7

        dX = growth_rate * X[t] * growth_factor
        X[t + 1] = max(0.0, X[t] + dX)

        # 4.2 Environmental load E
        # Driven by X, with simple linear decay.
        dE = env_sensitivity * X[t] - env_decay * E[t]
        E[t + 1] = max(0.0, E[t] + dE)

        # 4.3 Adaptive capacity C
        # Improves under moderate conditions, eroded under high stress,
        # with a small fatigue term.
        dC = 0.0
        if moderate_band:
            # Growth when the system is "not too calm, not too stressed"
            dC += adapt_gain * (1.0 - C[t])
        if high_stress:
            # Extra damage when stress is high
            dC -= stress_sensitivity * (E[t] - safe["E_max"])
        # Slow fatigue pulling C down
        dC -= fatigue * C[t]

        C[t + 1] = max(0.0, C[t] + dC)

    return {"X": X, "E": E, "C": C}


def compute_tension_metrics(E, C, safe):
    """
    Compute tension metrics for one trajectory.

    Metrics:
        - time_outside_safe: fraction of time steps outside the safe region.
        - max_distance: maximum distance to the safe region rectangle.
        - max_distance_norm: max_distance normalized by the rectangle diagonal.
        - boundary_speed_norm: normalized maximum change in distance when
          near the boundary.
        - T_anthro: weighted sum of the above components.
    """
    E = np.asarray(E)
    C = np.asarray(C)

    # Boolean mask for points inside the safe rectangle
    in_safe = (
        (E >= safe["E_min"]) & (E <= safe["E_max"]) &
        (C >= safe["C_min"]) & (C <= safe["C_max"])
    )
    time_outside_safe = 1.0 - in_safe.mean()

    # Distance to the safe rectangle in (E, C)
    E_clamped = np.clip(E, safe["E_min"], safe["E_max"])
    C_clamped = np.clip(C, safe["C_min"], safe["C_max"])

    distances = np.sqrt((E - E_clamped) ** 2 + (C - C_clamped) ** 2)

    # Normalize distances by the diagonal of the safe region
    diag_len = np.sqrt(
        (safe["E_max"] - safe["E_min"]) ** 2 +
        (safe["C_max"] - safe["C_min"]) ** 2
    )
    if diag_len > 0:
        max_distance = float(distances.max())
        max_distance_norm = float(max_distance / diag_len)
    else:
        max_distance = 0.0
        max_distance_norm = 0.0

    # Boundary speed: how sharply the trajectory moves when near the boundary
    if diag_len > 0:
        epsilon = 0.10 * diag_len  # "near boundary" band
        near_boundary = distances <= epsilon

        if len(distances) > 1:
            delta_dist = np.abs(np.diff(distances))
            # Consider steps where either endpoint is near the boundary
            mask = near_boundary[:-1] | near_boundary[1:]
            if mask.any():
                boundary_speed = float(delta_dist[mask].max())
                boundary_speed_norm = float(boundary_speed / diag_len)
            else:
                boundary_speed_norm = 0.0
        else:
            boundary_speed_norm = 0.0
    else:
        boundary_speed_norm = 0.0

    # Aggregate into a single tension score
    w1, w2, w3 = 0.5, 0.3, 0.2
    T_anthro = (
        w1 * time_outside_safe
        + w2 * max_distance_norm
        + w3 * boundary_speed_norm
    )

    return {
        "time_outside_safe": float(time_outside_safe),
        "max_distance": float(max_distance),
        "max_distance_norm": float(max_distance_norm),
        "boundary_speed_norm": float(boundary_speed_norm),
        "T_anthro": float(T_anthro),
    }


# -----------------------------
# 5. Run simulations
# -----------------------------

trajectories = {}
rows = []

for sc in scenarios:
    sid = sc["id"]
    sname = sc["short_name"]
    print(f"Simulating scenario: {sname} ({sc['name']})")

    traj = simulate_scenario(sc["params"], safe_region, n_steps=T_STEPS)
    trajectories[sid] = traj

    metrics = compute_tension_metrics(traj["E"], traj["C"], safe_region)

    row = {
        "scenario_id": sid,
        "scenario_short": sname,
        "scenario_name": sc["name"],
        "description": sc["description"],
        "time_outside_safe": metrics["time_outside_safe"],
        "max_distance": metrics["max_distance"],
        "max_distance_norm": metrics["max_distance_norm"],
        "boundary_speed_norm": metrics["boundary_speed_norm"],
        "T_anthro": metrics["T_anthro"],
    }
    rows.append(row)

print("\nAll simulations completed.\n")


# -----------------------------
# 6. Summary table
# -----------------------------

df = pd.DataFrame(rows)
df_sorted = df.sort_values("T_anthro", ascending=True).reset_index(drop=True)

print("Summary table (sorted by T_anthro, lower means closer to safe region):")
print(df_sorted[[
    "scenario_short",
    "scenario_name",
    "time_outside_safe",
    "max_distance_norm",
    "boundary_speed_norm",
    "T_anthro",
]])
print("")

print("Quick interpretation:")
for _, r in df_sorted.iterrows():
    print(
        f"  - {r['scenario_short']}: T_anthro ≈ {r['T_anthro']:.3f}, "
        f"time outside safe ≈ {r['time_outside_safe']:.2f}"
    )
print("")


# -----------------------------
# 7. Plots in (E, C) plane
# -----------------------------

fig, ax = plt.subplots(figsize=(7, 5))

# Draw safe region rectangle
rect = Rectangle(
    (safe_region["E_min"], safe_region["C_min"]),
    safe_region["E_max"] - safe_region["E_min"],
    safe_region["C_max"] - safe_region["C_min"],
    linewidth=1.5,
    edgecolor="gray",
    facecolor="none",
    linestyle="--",
)
ax.add_patch(rect)

# Plot trajectories
for sc in scenarios:
    sid = sc["id"]
    sname = sc["short_name"]
    traj = trajectories[sid]
    ax.plot(traj["E"], traj["C"], label=sname)

ax.set_xlabel("Environmental load E")
ax.set_ylabel("Adaptive capacity C")
ax.set_title("TU Q098_A · Trajectories in (E, C)")
ax.legend(title="Scenario")

# Set limits with a small margin
all_E = np.concatenate([trajectories[sc["id"]]["E"] for sc in scenarios])
all_C = np.concatenate([trajectories[sc["id"]]["C"] for sc in scenarios])
ax.set_xlim(min(all_E.min(), safe_region["E_min"]) - 0.1, max(all_E.max(), safe_region["E_max"]) + 0.1)
ax.set_ylim(min(all_C.min(), safe_region["C_min"]) - 0.1, max(all_C.max(), safe_region["C_max"]) + 0.1)

plt.tight_layout()
plt.show()

fig.savefig("Q098A_trajectories_E_C.png", dpi=150, bbox_inches="tight")
print("Saved trajectory plot as: Q098A_trajectories_E_C.png")


# -----------------------------
# 8. Bar plot of T_anthro
# -----------------------------

fig2, ax2 = plt.subplots(figsize=(6, 4))

ax2.bar(df_sorted["scenario_short"], df_sorted["T_anthro"])
ax2.set_xlabel("Scenario")
ax2.set_ylabel("T_anthro (higher = more tension)")
ax2.set_title("TU Q098_A · T_anthro per scenario")

plt.tight_layout()
plt.show()

fig2.savefig("Q098A_T_anthro_bar.png", dpi=150, bbox_inches="tight")
print("Saved tension bar plot as: Q098A_T_anthro_bar.png")


# -----------------------------
# 9. Pointer back to the main repo
# -----------------------------

print("\nWhere to go next:")
print("  - Main WFGY repository: https://github.com/onestardao/WFGY")
print("  - TU Event Horizon entry: https://github.com/onestardao/WFGY/blob/main/TensionUniverse/EventHorizon/README.md")
print("\nEnd of TU Q098_A offline MVP run.")
